# A Pythonic Object

## Vector Classs

In [1]:
from array import array
import math

In [2]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}{!r}, {!r}'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

> typecode是类属性，在Vector2d实例和字节序列之间转换时使用

## An Alternative Constructor

```python
@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)
```

## classmethod and staticmethod

classmethod定义操作类方法，而不是操作实例的方法。classmethod改变了调用方法的方式，因此类方法的第一个参数时类本身，而不是实例。classmethod最常见的用途是定义备选构造方法。

staticmethod装饰器也会改变调用方式，但是第一个参数不是特殊值。静态方法就是普通的函数，只是放在类的定义体中。

## Formatted Display

[PEP Advanced String Formatting](https://www.python.org/dev/peps/pep-3101/)

格式规范语言为一些内置类型提供了专用的表示代码。比如，b和x分别表示二进制和十六进制的int类型，f表示小数形式的float类型，而%表示百分数形式。

In [9]:
help(format)

Help on built-in function format in module builtins:

format(value, format_spec='', /)
    Return value.__format__(format_spec)
    
    format_spec defaults to the empty string.
    See the Format Specification Mini-Language section of help('FORMATTING') for
    details.



In [7]:
format(42, 'b')

'101010'

In [12]:
format(2/3, '.2%')

'66.67%'

格式规范语言是扩展的，因为各个类可以自行决定如何解释format_spec参数。例如：datetime模块中类，它们的`__format__`方法使用的格式嗲吗与strftime()函数一样。

In [32]:
import datetime

now = datetime.datetime.now()
format(now, '%H%:%M:%S')

'16:45:13'

In [35]:
"It's now {:%I:%M %p}".format(now)

"It's now 04:45 PM"

如果类没有定义`__format__`方法，从object继承的方法会返回str(my_object)。我们为Vector2d定义了`__str__`方法：

In [36]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

然而，如果传入格式说明符，`object.__format__`方法会抛出TypeError:

In [38]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

```python
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({}, {})'.format(*components)
```

## A Hashed Vector2d

注意，我们让这些向量不可变是有原因的，因为这样才能实现`__hash__`方法。这个方法应该返回一个整数，理想情况下还要考虑对象属性的散列值(`__eq__`方法也要使用)，因为相等的对象应该具有相同的散列值。根据特殊方法`__hash__`的文档，最好使用位运算符异或（^）混合各分量的散列值。

```python
def __hash__(self):
    return hash(self.x) ^ hash(self.y)
```

> 要想创建可散列的类型，不一定要实现特性，也不一定保护实例属性。只需正确的实现`__hash__`和`__eq__`方法即可。但是，实例的散列值绝不应该变化。

In [50]:
from array import array
import math

In [51]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = x
        self.__y = y
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
            
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

## Private and Protected Attributes in Python

In [52]:
v = Vector2d(1, 2)

In [55]:
v.__dict__

{'_Vector2d__x': 1, '_Vector2d__y': 2}

Python不像Java那样使用private修饰符创建私有属性，但是Python有个简单的机制能避免子类覆盖“私有”属性。

如果以`__mod`的形式命名实例属性，Python会把属性名存入实例的`__dict__`属性中，而且会在前面添加一个下划线和类名。

In [21]:
class Foo:
    
    def __init__(self):
        self.__x = 'x'
        self.__y = 'y'

In [60]:
ins = Foo()

In [61]:
ins.__dict__

{'_Foo__x': 'x', '_Foo__y': 'y'}

## Saving Space with the `__slots__` Class Attribute

默认情况下，Python在各个实例中名为`__dict__`的字典中存储实例属性。为了使底层的散列表提升访问速度，字典会消耗大量的内存。如果要处理数百万个属性的实例，通过`__slots__`类属性，能节省大量内存，方法是让解释器在元组中存储实例属性，而不用字典。

By default, Python stores instance attribtes in a per-instance dict name `__dict__`. Dinctionaries have a significant memory overhead because of the underlying hash table used to provide fast access. If you are dealing with millions of memory, by letting the interpreter store the instance attribtutes in a tuple instead of a dict.

> 继承自超类的`__slots__`属性没有效果。Python只会使用各个类中定义的`__slots__`属性。

> A `__slots__` attribute inherited from a superclass has no effect. Python only takes into account `__slots` attributes defined in each class individually.

定义`__slots__`的方式是，创建一个类属性，使用`__slots__`这个名字，并把它的值设为一个字符串构成的可迭代对象，其中各个元素表示各个实例属性。我喜欢使用元组，因为这样定义的`__slots__`中所含的信息不会变化。

In [62]:
class Vector2d:
    
    __slots__ = ('__x', '__y')
    typecode = 'd'

在类中定义`__slots__`属性的目的是告诉解释器：“这个类中的所有实例属性都在这了！”。这样，Python会在各个实例中使用类似元组的结构存储实例变量，从而避免使用消耗内存的`__dict__`属性。如果有数百万个实例同时活动，这样做能节省大量内存。

> 如果要处理数百万个数值对象，应该使用NumPy数组。NumPy数组能高效使用内存，而且提供了高度优化的数值处理函数，其中很多都一次操作整个数组。

### The Problem with `__slots__`

`__slots__`能显著节省内存，不过有几点要注意：

* 每个子类都要定一个`__slots__`属性，因为解释器会忽略继承的`__slots__`属性
* 实例中只能拥有`__slots__`中列出的属性，除非把`__dict__`加入`__slots__`中（这样做就失去了节省内存的功效）
* 如果不把`__weakref__`加入`__slots__`，实例就不能作为弱引用的目标

## Overriding Class Attributes

Python有个很独特的特性：类属性可用于为实例属性提供默认值。Vector2d中有个typecode类属性，`__bytes__`方法两次用到它，使用`self.typecode`读取它的值。因为Vector2d实例本身没有typecode属性，所以`self.typecode`默认获取的是`Vector2d.typecode`类属性的值。

## Reference

* [Advanced String Formatting](https://www.python.org/dev/peps/pep-3101/)